In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

#core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Read the data and explore it

In [ ]:
vehicle=pd.read_csv('../input/vehicle-dataset-from-cardekho/car data.csv')
vehicle.head()

In [ ]:
## check how many records and features are in the dataset
vehicle.shape

In [ ]:
vehicle.info()

In [ ]:
## check missing values
vehicle.isnull().sum()

### As there are no missing values, it's good to progress to next steps

In [ ]:
# see unique values of categorical features
print(vehicle['Fuel_Type'].unique())
print(vehicle['Seller_Type'].unique())
print(vehicle['Transmission'].unique())
print(vehicle['Owner'].unique())

#### Here, we will drop 'Car_Name' column from our dataset as it is not required to predict the selling price of a car.

In [ ]:
car_dataset = vehicle.drop(['Car_Name'],axis=1)

In [ ]:
car_dataset.head()

In [ ]:
car_dataset.columns

#### We have to create a new feature 'Current_Year' as 2021 and find 'Total_years' so that we can see how much old is our vehicle. 

#### As the vehicle is getting older, the selling price is also decreased.

In [ ]:
car_dataset['Current_Year']=2021

In [ ]:
car_dataset.head()

In [ ]:
car_dataset['Total_Years']= car_dataset['Current_Year']-car_dataset['Year']

In [ ]:
car_dataset.head()

####  We have to drop 'Year' and 'Current_Year' . Year is basically the purchased year and we dont need it. The selling price prediction didnt need year and current year but it needs total years which we have already calculated.

In [ ]:
car_dataset.drop(['Year','Current_Year'],axis=1,inplace=True)

In [ ]:
car_dataset.head()

#### We have to convert categorical features to one hot encoded values because we have less number of features and dropping the first feature to avoid dummy variable trap.

In [ ]:
car_dataset=pd.get_dummies(car_dataset,drop_first=True)

In [ ]:
car_dataset.head()

In [ ]:
## By visualizing the data in pairplot, we didnt get that much information.
sns.pairplot(car_dataset)

#### Let's see the co-related features info with the help of HEATMAP

In [ ]:
## Here, we are storing the dataset co-related features index values in a variable 'cor_mat' so as to use that values in heatmap.
## Index values like Selling_price,present_price,kms_driven.....etc,etc
cor_mat=car_dataset.corr().index

In [ ]:
# Plotting the heatmap 
sns.heatmap(car_dataset[cor_mat].corr(),annot=True,cmap='RdYlGn')

#### Here, Fuel_type_Petrol and Fuel_Type_diesel is -vely co-related(red color box)

In [ ]:
car_dataset.head()

In [ ]:
## Independent features and dependent features
X=car_dataset.iloc[:,1:]
y=car_dataset.iloc[:,0]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [ ]:
## Hyperparameters
n_estimators=[int(x) for x in np.linspace(start=100,stop=1200,num=12)]
print(n_estimators)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_random.fit(X_train,y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_score_

In [ ]:
predictions=rf_random.predict(X_test)

In [ ]:
sns.histplot(y_test-predictions,kde=True)

In [ ]:
plt.scatter(y_test,predictions)